# 仮説検定（Hypothesis Testing）

### 平均値の検定　（母分散が未知） T検定
$$
T = \frac{\hat{\mu} - \mu_0}{\hat{\sigma}/\sqrt{N}}
$$
これは自由度$ df = N - 1 $の$t$分布に従う（ $df$: Degree of Freedom）<br>
scipy.stats.t https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html <br>

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline

#### 例１：あるクラス，テストの平均点と補講の効用（片側検定）

In [ ]:
data = np.array([1, -1, -2, 3, -1, 5, 4, 0, 7, -1])
mean, std = np.average(data) , np.std(data, ddof=1) # std, ddof=1 : unbiased
N = len(data) # the number of sample
print(mean, std, N)

In [ ]:
alp = 0.05
# pp: percent point, isf (Inverse survival function) 
x_alp = stats.t.isf(alp,N-1) # 上側パーセント点,
print('x_alp (alpha=%5.2f, df=%d) =%f' %(alp, (N-1),x_alp))

mu = 0 # null hypothesis
T = (mean-mu)/(std/np.sqrt(N))
print('T=', T)

x_alp > T より，H0は棄却できない。
通常は，このようなパーセント点を求めるよりは，p値(p value)を求めて検定する。これを次に示す。

In [ ]:
prob = stats.t.cdf(T,N-1)
prob2 = stats.t.sf(T,N-1)
print('p value=',1-prob, prob2)

有意水準よりp valueの方が大きいので，H0を棄却できない。

#### 例2：精密部品の直径（両側検定）
$\alpha = 0.05$とする。

In [ ]:
data2 = np.array([1.5399, 1.5390, 1.5399, 1.5395, 1.5400, 1.5390, 1.5399, 1.5399])
mean, std = np.mean(data2), np.std(data2, ddof=1)
N = len(data2)
mu = 1.54
print('sample mean =',mean,'  std = ',std,' The number of data = ',N)

In [ ]:
T = (mean-mu)/(std/np.sqrt(N))
prob = stats.t.cdf(T, df=N-1)
if T >=  0:
    p = 1 - prob
else:
    p = prob

print('T = ',T, 'p value =',p)

注意：上記のように，Tが負の値をとるときは，p = prob とする。  
両側検定ゆえ$\alpha/2=0.025$を考えて，p value $< \alpha/2$ よりH0を棄却

次の関数を用いる方法を紹介する。<br>
T, p = scipy.stats.ttest_1samp(data, mu)<br>
data: 1群のサンプルデータ<br>
mu: 帰無仮説H0で仮定した平均値<br>
T: t値<br>
p: p値，両側検定を前提としている。片側検定ではこの半分の値を用いる<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html

In [ ]:
mu=1.54
T, p = stats.ttest_1samp(data2, mu)
print('T = ',T) # 一方のパーセント点のみ
print('two tailed p = ',p)
print('one tailed p = ',p/2)
print('df =', len(data2)-1)

## 分散の検定　（母平均が未知） $\chi ^2$検定
$$
X = \frac{N-1}{\sigma ^2} \hat{\sigma}^2 \sim \chi^2 (N-1)
$$
上記の$\chi^2$検定量は自由度$ df = N - 1 $の$\chi^2$分布に従う（ $df$: Degree of Freedom）<br>
scipy.stats.chi2 https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html <br>
$\chi^2$分布は非対称，かつ，正の領域の分布形状である。

#### 例３：先の例の精密部品の直径のばらつき（分散）の検定
分散の検定ゆえに片側検定である。

In [ ]:
uvar = np.var(data2, ddof=1) # unbaiased
var, N = 1.e-7, len(data2)

ratio = (uvar / var)
X = (N-1)*ratio
print('unbaiased variance =',uvar,'  var =', var, '  (uvar/var)', ratio)

pval = stats.chi2.sf(X, df=N-1)
print('p value=',pval)

$\alpha = 0.05$とすれば，p value $> \alpha$ より，H0を棄却できない。  
平均値の検定で，平均値がずれているという結論だった。分散の検定でH0を棄却できていないので，事実とはまだ言えないが，  
この部品は高精度であるが，低正確度（Wikipedia参照）の可能性がある。<br>
https://ja.wikipedia.org/wiki/正確度と精度

## 2標本の平均値の差の検定

2標本，両方の母分散が共に未知である場合を扱う。この場合ウェルチのt検定（Welch's t-test）を用いる。  
統計検定量$t$は複雑ゆえ，次を参照されたい：https://en.wikipedia.org/wiki/Welch%27s_t-test  
この自由度は，ウェルチ-サタスウェイトの式（Welch–Satterthwaite equation）より近似的に求められる。
https://en.wikipedia.org/wiki/Welch%E2%80%93Satterthwaite_equation  

自由度も自動的に計算する関数が次のscipy.stats.ttest_ind,  入力で equal_var = False を指定する  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

#### 例4：2つの温度計の性能検定（対応の無い2標本，分散が未知）

In [ ]:
s1 = np.array([37.1, 36.7, 36.6, 37.4, 36.8, 36.7, 36.9, 37.4, 36.6, 36.7])
s2 = np.array([36.8, 36.6, 36.5, 37.0, 36.7, 36.5, 36.6, 37.1, 36.4])

In [ ]:
t, p = stats.ttest_ind(s1, s2, equal_var=False)
print('t = ',t, ' p value = ',p)

p value を見て，alpha = 5% より大きいのでH0を棄却できない。すなわち，2つの体温計の平均値が等しいという仮説は棄却できない。

参考までに，自由度の計算の仕方を紹介する。  
ウェルチ=サタスウェイトの式により近似自由度を計算する   
https://en.wikipedia.org/wiki/Welch%27s_t-test

In [ ]:
s1m = s1.mean()
s1v = s1.var(ddof=1)
s2m = s2.mean()
s2v = s2.var(ddof=1)

bunshi = ((s1v/len(s1)) + (s2v/len(s2)))**2
bunbo = (s1v/len(s1))**2/(len(s1)-1) + (s2v/len(s2))**2/(len(s2)-1)
nu = bunshi/bunbo
nu

自由度は実数となる。本来，一番近い整数を与えるのだが，数値的には計算できるので，この自由度をそのまま与える。

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html

In [ ]:
prob = stats.t.cdf(t, df=nu)
p = 1-prob
print(2*p)  # 両側検定とすれば，2*pとなる

この結果は，先の p valueとほぼ一致する

## 相関と無相関の検定
相関係数$\hat{\rho}$には次の性質がある。
$$
- 1 \le \hat{\rho}  \le 1
$$
統計検定量は次を用いる
$$
t = \frac{{ |\hat \rho _{xy} | \sqrt {N - 2} }}{{\sqrt {1 - \hat \rho _{xy} ^2 } }} \sim t(N-2)
$$
これは，自由度$(N-2)$の$t$分布に従う

ピアソンの相関係数と帰無仮説(無相関)と設定した場合のp値　の両方を同時に計算する  
scipy.stats.pearsonr  
https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.pearsonr.html

#### 例5：親子の身長の相関の検定

In [ ]:
x = np.array([168, 172, 181, 179, 166, 185, 177, 176, 169, 161])
y = np.array([111, 125, 129, 120, 126, 133, 130, 116, 118, 115])
corr, pvalue = stats.pearsonr(x,y)
print('corr =',corr, '  p value=',pvalue)

alp = 0.05 のとき，p value < alp より，H0（無相関）を棄却，alp = 0.01のとき棄却できない

上記のstats.pearsonr()がどのようにpvalueを計算しているかを確かめたのが次である。<br>
計算結果をみて，stats.pearsonr()はp値を次の計算を用いていることが分かる。

In [ ]:
t = np.abs(corr)*np.sqrt(len(x)-2)/(np.sqrt(1-corr**2))
p = 2*stats.t.sf(t, df=len(x)-2)
p

#### 上記のデータの散布図

In [ ]:
plt.scatter(x,y, color='k')
plt.xlabel('Height of father', fontsize=14)
plt.ylabel('Height of son', fontsize=14)
#plt.savefig('fig_Stat_CorrelationTest.pdf')